In [1]:
import os
import cv2

import os
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.misc import imsave
from imageio import imwrite
from numpy.random import randint
from keras.models import load_model,save_model

os.chdir('/home/rsk/Documents/Projects/YOLO/BoundingBox')

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_size = 25,25,3

In [3]:
from numpy.random import randint

def createBlock(x,y,width,height):
    
    img = np.ones(img_size)
    img[:,:,:]=255

    
    img[x:(x+width), y:(y+height),:]=0
    
    return([img,[x,y,width,height]])

In [4]:
x,y = 12,12
width,height = 6,6
numImages = 1000

In [5]:
imgCount = 0
movementCount=0
os.chdir("/home/rsk/Documents/Projects/YOLO/BoundingBox/VideoImageFolder/")

while(True):
    
    if movementCount%3==0:
        movement = randint(1,5)
        
    
    if movement ==1 :        
        if y-1<0:
            continue
        else:
            y = y-1
            
    elif movement ==2:
       
        if x+width+1>=img_size[0]:
            continue
        else:
            x = x+1
        
    elif movement ==3:
        
        if y+height+1>=img_size[1]:
            continue
        else:
            y = y+1
    else:
        if x-1<0:
            continue
        else:
            x = x-1
            
            
    movementCount+=1
    
    img,[x,y,width,height] = createBlock(x,y,width,height)
    img = img.astype(np.uint8)
    
    imwrite('img'+str(imgCount)+'.png',img)
    
    imgCount+=1
    if imgCount ==numImages:
        break

    
        
        

KeyboardInterrupt: 

In [6]:
# Saving the images into a video file
os.chdir("/home/rsk/Documents/Projects/YOLO/BoundingBox/VideoImageFolder/")
os.system("ffmpeg -r 10 -i img%d.png -vcodec mpeg4 -y 50x50_1000_movie.mp4")

0

### Testing Model on Video File

In [7]:
def testFrame(frame,model):
    
    
    img = np.reshape(frame,(1,25,25,3))
    img = np.array(img)
    img = img/255.0
    
    pred = model.predict(img)
    checkLimit = lambda x : min(max(0.0,x),25.0)
    
    pred = [checkLimit(pred[0][0]) , checkLimit(pred[0][1]), checkLimit(pred[0][2]), checkLimit(pred[0][3])]
    return(pred)
    

In [8]:
model = load_model('/home/rsk/Documents/Projects/YOLO2/Blocks/Models/model_3_143_0.03.hdf5')

In [9]:
cap = cv2.VideoCapture('/home/rsk/Documents/Projects/YOLO2/Blocks/25x25_1000_movie.mp4')
frameCount =0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret==True:

        # Our operations on the frame come here
#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        frame = cv2.resize(frame,(25,25))
        pred = testFrame(frame,model)
        
        #To be used for testing purposes only
#         if frameCount == 10:
#             break    
#         frameCount+=1
        
        #Plotting the predicted boundary box
        cv2.rectangle(frame,(int(pred[1]),int(pred[0])) , (int(pred[1]+pred[3]),int(pred[0]+pred[2])),(0,255,0))
        

#         frame = cv2.resize(frame,(100,100))
        
        # Display the resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()